In [1]:
from pandas_datareader import data as pdr
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
start = (datetime.datetime.today() - datetime.timedelta(days=366)).strftime('%m/%d/%Y')
end = datetime.datetime.today().strftime('%m/%d/%Y')
    
df = pdr.get_data_yahoo('GOOG', start=start, end=end)

In [19]:
def StockVol(histoPrice):
    closing_prices = histoPrice['Adj Close'].values
    returns = np.log(closing_prices[1:]/closing_prices[:-1])
    vol = np.sqrt(252)*np.sqrt(np.var(returns))
    return vol

The StockVol function simply calculates the historical volatility of a stock.

In [4]:
def StockPath(n, sigma, S0, T, num_p, r):
    paths = np.zeros((n, num_p+1))
    paths[:, 0] = S0
    dt = T/num_p
    
    for i in range(n):
        for j in range(1, num_p+1):
            rand = np.random.standard_normal()
            paths[i, j] = paths[i, j-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*rand)
    
    return paths

The StockPath function generates n paths which we will use to simulate potential payoff, allowing us to price options.

In [5]:
def EurOptPrice(n, paths, sigma, T, num_p, r, K):
    payoff = np.exp(-r*T)*np.maximum(K-paths[:, num_p], 0)
    price = np.average(payoff)
    var = np.var(payoff)
    return [payoff, price, var]

The EurOptPrice function averages each paths inflation adjusted terminal put payout to price a European put.

In [99]:
def AmeOptPrice(n, paths, sigma, T, num_p, r, K):
    V_hat = np.zeros((n, num_p))
    h = np.maximum(K-paths, 0)
    V_hat[:, num_p-1] = h[:, num_p-1]

    for i in range(num_p-1, 0, -1):
        fit = np.polyfit(paths[:, i], V_hat[:, i]*np.exp(-r*T/num_p), 5)
        fitted = np.polyval(fit, paths[:, i])
        V_hat[:, i-1] = np.where(h[:, i] > fitted, h[:, i], V_hat[:, i]*np.exp(-r*T/num_p))

    price = np.mean(V_hat[:, 1]*np.exp(-r*T/num_p))
    var = np.var(V_hat[:, 1]*np.exp(-r*T/num_p))
    return [V_hat, price, var]

The AmeOptPrice function uses polynomial regression to evaluate the optimal time to excerice an American put. If the regression fitted value is greater than the payoff at terminal we will exercise at the time, allowing us to achieve optimal value and thus properly pricing an American put.

In [179]:
def ContVariate(n, paths, sigma, T, num_p, r, K):
    V_hat = AmeOptPrice(n, paths, sigma, T, num_p, r, K)[0]
    X = EurOptPrice(n, paths, sigma, T, num_p, r, K)[0]
    
    dt = T/num_p
    df = np.exp(-r*dt)
    
    Y = V_hat[:, 1]*df
    
    EX = np.average(X)
    
    X_mean = np.average(X)
    Y_mean = np.average(Y)
    b = np.sum((X-X_mean)*(Y-Y_mean))/np.sum((X-X_mean)**2)
    estimator = (1/n)*np.sum(Y-b*(X-EX))
    var = np.var(Y-b*(X-EX))
    return [estimator, var]

The ContVariate function uses the simulated known quantity of the European put payoff of each path to reduce variance of the American put price. The simulated European payoff minus the expectation of payoff for each path is an unbiased estimation. Using the correlation of this unbiased estimator and the value we are trying to predict we can calculate the optimal coefficient b. This optimal coefficient b best reduces variance, providing a more relaible estimate for option pricing. 

In [180]:
vol = StockVol(df)
print("Google stock has historical volatility " + str(vol))
S0 = df['Adj Close'][0]
T = 1
num_p = 252
r = 0.0068
n = 1000

paths = StockPath(n, vol, S0, T, num_p, r)

K = 1440
priceEuro = EurOptPrice(n, paths, vol, T, num_p, r, K)[1:]
print("The price and variance, respectively, of the European put option with strike " + str(K) + " is: " + str(priceEuro))

priceAmer = AmeOptPrice(n, paths, vol, T, num_p, r, K)[1:]
print("The price and variance, respectively, of the American put option with strike " + str(K) + " is: " + str(priceAmer))

price_cont_var = ContVariate(n, paths, vol, T, num_p, r, K)
print("The price and variance, respectively of the American put option with strike " + str(K) + " using control variates as "
      "a technique for variance reduction is: " + str(price_cont_var))

Google stock has historical volatility 0.3593221344992161
The price and variance, respectively, of the European put option with strike 1440 is: [397.4381360116953, 83591.35917968172]
The price and variance, respectively, of the American put option with strike 1440 is: [409.91172061527647, 50058.91812604502]
The price and variance, respectively of the American put option with strike 1440 using control variates as a technique for variance reduction is: [409.91172061527647, 22785.85369909315]


This is the best way to price an American option because it both evaluates the best potential return by utilizing polynomial regression to optimize exercise time and implements a control variate to reduce variance, allowing us to make an accurate and reliable estimate. 